# SMU Logo bounding box detection
Design an algorithm to identify all images of SMU from a set of random images. A bounding box should be drawn around the SMU logo whenever it appears in an image.

When an SMU logo occurs in an image, a bounding box should be drawn around it. Logo identification will be evaluated by its F1 score.

### Setup
Install dependencies
```bash
pip install -r requirements.txt

```

In [ ]:
%pip install -r requirements.txt

Import packages

In [1]:
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
from dotenv import load_dotenv
from manage_model import update_path, convert_bbox_format, get_f1
import os

Load environment variables using dotenv
- `API_KEY` - API Key for Roboflow dataset
- `DATA_VERSION` - Version of dataset used
- `PATH_TO_DATA` - Absolute file path to the "SMU-Logo-Detection-1" folder. See .env.example for example

In [2]:
# Project settings
load_dotenv()
api_key = os.getenv('API_KEY')
data_version = os.getenv('DATA_VERSION')
path_to_project = os.getenv("PATH_TO_PROJECT")

# If on SMU Violet, can't load env variables. Write manually.

path_to_data = f'{path_to_project}/tree-type-detection-{data_version}'

Load dataset into workspace

In [3]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("tree-dataset-iftyz").project("tree-type-detection-9rfxy")
dataset = project.version(data_version).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


Update test, train and val values in data.yaml
This fixes a file not found bug

In [4]:
update_path(data_version, path_to_project)

### [Training](https://docs.ultralytics.com/modes/train/#train-settings)
Use a trained model, saved in `/saved_models`, or train a model.
A pre-trained [model](https://github.com/ultralytics/ultralytics?tab=readme-ov-file) `yolov8n.pt` from Ultralytics should be used. Tune hyper-params such as learning rate and epochs.

If there is file not found error, check the `FILE_TO_PATH` env variable, and make sure that you've updated the `data.yml` to the absolute file path.

In [ ]:
# # Train model (Laptop)
# model = YOLO('yolov8n.pt') # pre-trained yolov8 nano
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10)

# # Train model (SMU GPU)
# model = YOLO('yolov8n.pt')
# results = model.train(data=f'{path_to_data}/data.yaml', epochs=10, batch=-1, device=0, workers=4)

In [5]:
# Use saved model
model = YOLO('./saved_models/v4_epoch13.pt')

After training, the model can be found in `/runs`. Save the `best.pt` model in the `/saved_models` folder.

### [Validation](https://docs.ultralytics.com/modes/val/#usage-examples)
Val mode is used for validating a YOLOv8 model after it has been trained. In this mode, the model is evaluated on a validation set to measure its accuracy and generalization performance. This mode can be used to tune the hyperparameters of the model to improve its performance.

In [6]:
validation_results = model.val(data=f'{path_to_data}/data.yaml', conf=0.25, iou=0.6)
'''
1. Epochs: Vary from 7, 15 etc
2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics
3. Re-annotate if have time (those with very small logos)


4. annotate the rest of the images
5. after everything is annotated, augmentation and final training
'''

Ultralytics YOLOv8.0.196  Python-3.10.6 torch-2.2.1+cpu CPU (AMD Ryzen 9 5900HX with Radeon Graphics)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\valid\labels.cache... 576 images, 463 backgrounds, 0 corrupt: 100%|██████████| 576/576 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:50<00:00,  1.41s/it]
                   all        576        129      0.947      0.729      0.855      0.691
Speed: 1.5ms preprocess, 78.7ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\val4


'\n1. Epochs: Vary from 7, 15 etc\n2. Find a way to output the predicted labels into a folder. Compare the predicted labels and actual labels to check the metrics\n3. Re-annotate if have time (those with very small logos)\n\n\n4. annotate the rest of the images\n5. after everything is annotated, augmentation and final training\n'

In [7]:
get_f1(validation_results.results_dict)

0.8234583741466337

### [Test](https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference)
Predict bounding boxes on test set

In [8]:
# Predict
prediction = model.predict(source=f"{path_to_data}/test/images", conf=0.25, iou=0.6)


image 1/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8053_JPG.rf.2d2db88e7f26cda9e376dbafe963a9d9.jpg: 640x640 (no detections), 163.0ms
image 2/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8067_JPG.rf.730df81b052e107a9fce34b09b875eb6.jpg: 640x640 (no detections), 110.0ms
image 3/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8075_JPG.rf.0280008f3f6e1d5c33599c629843eeed.jpg: 640x640 (no detections), 91.0ms
image 4/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8090_JPG.rf.fdacda6e11509d87bb5fc07923be1c36.jpg: 640x640 (no detections), 91.0ms
image 5/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG_8165_JPG.rf.fc2fd99a3964df0741ace08abf6c88fe.jpg: 640x640 (no detections), 92.0ms
image 6/288 C:\Users\jinha\Documents\CS424\Project SMU Logo\tree-type-detection-4\test\images\IMG

Save images to `/predictions` folder if bounding box exists.

In [12]:
images = {}

if not os.path.exists(f'{path_to_data}/../predictions'):
    os.makedirs(f'{path_to_data}/../predictions')

for i in range(0, len(prediction)):
    if prediction[i].boxes.shape[0] >= 1:
        im_rgb = Image.fromarray(prediction[i].plot(line_width=1)[..., ::-1])
        images[i] = im_rgb  # RGB-order PIL image
        predicted_box = convert_bbox_format(prediction[i])

        # Write labels
        with open(f'{path_to_data}/../predictions/obj_est.txt', 'a') as f:
            f.write(', '.join(map(str, predicted_box)) + '\n')
        # Save images
        im_rgb.save(fp=f'{path_to_data}/../predictions/{predicted_box[0]}_labelled.jpg')

len(images)

54